In [6]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [8]:
X_new_scaled = X_test_scaled

In [93]:
class MySGDOptimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.001, momentum=0.9, name="MySGDOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._set_hyper("decay", self._initial_decay) # 
        self._set_hyper("momentum", momentum)
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "momentum")

    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # handle learning rate decay
#         momentum_var = self.get_slot(var, "momentum")
#         momentum_hyper = self._get_hyper("momentum", var_dtype)
#         momentum_var.assign(momentum_var * momentum_hyper - (1. - momentum_hyper)* grad)
        var.assign_sub(grad * lr_t)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "decay": self._serialize_hyperparameter("decay"),
            "momentum": self._serialize_hyperparameter("momentum"),
        }

In [94]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [96]:
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[8])])
model.compile(loss="mse", optimizer=MySGDOptimizer(learning_rate=0.001))
model.fit(X_train_scaled, y_train, epochs=20)

Train on 11610 samples
Epoch 1/20
11610/11610 [==============================] - 1s 81us/sample - loss: 3.9043
Epoch 2/20
11610/11610 [==============================] - 0s 38us/sample - loss: 1.5243
Epoch 3/20
11610/11610 [==============================] - 0s 37us/sample - loss: 1.0021
Epoch 4/20
11610/11610 [==============================] - 0s 37us/sample - loss: 0.8606
Epoch 5/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.8038
Epoch 6/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.7715
Epoch 7/20
11610/11610 [==============================] - 0s 39us/sample - loss: 0.7457
Epoch 8/20
11610/11610 [==============================] - 0s 40us/sample - loss: 0.7228
Epoch 9/20
11610/11610 [==============================] - 0s 40us/sample - loss: 0.7012
Epoch 10/20
11610/11610 [==============================] - 0s 41us/sample - loss: 0.6834
Epoch 11/20
11610/11610 [==============================] - 0s 43us/sample - loss: 0.6681
Epoch 1

In [102]:
class SGOptimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, name="SGOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._is_first = True
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "pv") #previous variable i.e. weight or bias
        for var in var_list:
            self.add_slot(var, "pg") #previous gradient


    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # handle learning rate decay
        new_var_m = var - grad * lr_t
        pv_var = self.get_slot(var, "pv")
        pg_var = self.get_slot(var, "pg")
        
        if self._is_first:
            self._is_first = False
            new_var = new_var_m
        else:
            cond = grad*pg_var >= 0
            print(cond)
            avg_weights = (pv_var + var)/2.0
            new_var = tf.where(cond, new_var_m, avg_weights)
        pv_var.assign(var)
        pg_var.assign(grad)
        var.assign(new_var)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

In [103]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [104]:
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[8])])
model.compile(loss="mse", optimizer=SGOptimizer(learning_rate=0.001))
model.fit(X_train_scaled, y_train, epochs=50)

Train on 11610 samples
Epoch 1/50
Tensor("GreaterEqual:0", shape=(1,), dtype=bool)
11610/11610 [==============================] - 1s 95us/sample - loss: 3.7333
Epoch 2/50
11610/11610 [==============================] - 1s 47us/sample - loss: 1.4848
Epoch 3/50
11610/11610 [==============================] - 1s 48us/sample - loss: 0.9218
Epoch 4/50
11610/11610 [==============================] - 1s 46us/sample - loss: 0.7634
Epoch 5/50
11610/11610 [==============================] - 1s 46us/sample - loss: 0.7067
Epoch 6/50
11610/11610 [==============================] - 1s 45us/sample - loss: 0.6801
Epoch 7/50
11610/11610 [==============================] - 1s 45us/sample - loss: 0.6624
Epoch 8/50
11610/11610 [==============================] - 1s 47us/sample - loss: 0.6482
Epoch 9/50
11610/11610 [==============================] - 1s 46us/sample - loss: 0.6354
Epoch 10/50
11610/11610 [==============================] - 1s 47us/sample - loss: 0.6252
Epoch 11/50
11610/11610 [======================

In [58]:
a = tf.Variable([-1,2,2,0])
b = tf.Variable([5,6,7,8])
cond = a*b >= 0
cond

<tf.Tensor: shape=(4,), dtype=bool, numpy=array([False,  True,  True,  True])>

In [59]:
tf.where(cond, a, b)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([5, 2, 2, 0], dtype=int32)>